* 基于xception 全部训练集的模型，生成预测结果 neckline_0419Xception_alldata.csv

In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import itertools

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 导入数据
-  label 文件一共有 79572 行
- 各种维度混合在一起
- 我们的目的是切分开各种维度, 进行训练和模拟
- 那么首先取出个标签的图片进行聚类

In [2]:
df_train = pd.read_csv('../data/base/Annotations/label.csv', header=None)
df_train.columns = ['image_id', 'class', 'label']
df_train.head()

,image_id,class,label
0,Images/collar_design_labels/4d8a38b29930a403e5...,collar_design_labels,nnynn
1,Images/collar_design_labels/bd0981f231180d2b00...,collar_design_labels,nynnn
2,Images/collar_design_labels/26937e1724feadfe39...,collar_design_labels,ynnnn
3,Images/collar_design_labels/cf4140ec542887270f...,collar_design_labels,nynnn
4,Images/collar_design_labels/50644b2b9de045f2d1...,collar_design_labels,nynnn


In [3]:
classes = ['collar_design_labels', 'neckline_design_labels', 'skirt_length_labels',
          'sleeve_length_labels', 'neck_design_labels', 'coat_length_labels', 'lapel_design_labels',
          'pant_length_labels']

In [5]:
cur_class = classes[1]
df_load = df_train[(df_train['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

neckline_design_labels: 17146


,image_id,class,label
0,Images/neckline_design_labels/709886d646acf9cb...,neckline_design_labels,nnynnnnnnn
1,Images/neckline_design_labels/849e4a6ac9fac7ed...,neckline_design_labels,nnnnnnynnn
2,Images/neckline_design_labels/03cb7abfd21ffb23...,neckline_design_labels,nnnnynnnnn
3,Images/neckline_design_labels/39ffaba42324462f...,neckline_design_labels,nnynnnnnnn
4,Images/neckline_design_labels/cda470cf1f539c2c...,neckline_design_labels,nnnnnynnnn


In [7]:
n = len(df_load)
n_class = len(df_load['label'][0])
width = 299 # 定义图片大小

X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, n_class), dtype=np.uint8)

In [8]:
n_class

5

In [10]:
n = len(df_load)
n_class = len(df_load['label'][0])
width = 299 # 定义图片大小

X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, n_class), dtype=np.uint8)

In [9]:
for i in tqdm(range(n)):
    tmp_label = df_load['label'][i]
    if len(tmp_label) > n_class:
        print(df_load['image_id'][i])
    X[i] = cv2.resize(cv2.imread('./{0}'.format(df_load['image_id'][i])), (width, width))
    y[i][tmp_label.find('y')] = 1

## 提取特征

In [11]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
import keras

In [12]:
cnn_model = Xception(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [13]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [6]:
prefix_cls = cur_class.split('_')[0]
prefix_cls

'neckline'

## 划分训练/测试集

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.12, random_state=42)
X_train.shape, y_train.shape

((6189, 299, 299, 3), (6189, 5))

In [16]:
# Compile the model
adam = Adam(lr=0.001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 16
batch_size = 32

## 数据增强

In [17]:
datagen = ImageDataGenerator(
        featurewise_center = False, # set input mean to 0 over the dataset
        samplewise_center = False, # set each sample mean to 0
        featurewise_std_normalization = False, # divide inputs by std of the dataset
        samplewise_std_normalization = False, # divide each input by its std
        zca_whitening = False, # apply ZCA whitening
        rotation_range = 10, # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # randomly zoom image
        width_shift_range = 0.1, # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.1, # randomly shift images vertivally (fraction of total heigth)
        horizontal_flip = True, # randomly flip images
        vertical_flip = False,
        shear_range = 0.1,
        fill_mode = 'constant',
        cval = 0)

datagen.fit(X_train)

In [10]:
prefix_cls = cur_class.split('_')[0]

checkpointer = ModelCheckpoint(filepath='./models/{0}.bestXception0419_alldata.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True, mode='val_acc')

try:
    # Fit the model
    history = model.fit_generator(datagen.flow(X, y, batch_size=batch_size),
                                  epochs=epochs, validation_data=(X_valid, y_valid),
                                  verbose=2, steps_per_epoch=X.shape[0] // batch_size,
                                  callbacks=[EarlyStopping(patience=5), checkpointer])
except KeyboardInterrupt:
    print('KeyboardInterrupt')

### 减小学习率 lr =0.0001 , 继续进行训练

In [19]:
cnn_model = Xception(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [20]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [21]:
model.load_weights('./models/{}.bestXception0419_alldata.h5'.format(prefix_cls)) #加载以前训练好的模型，继续测试

In [22]:
# Compile the model
adam = Adam(lr=0.0001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 16
batch_size = 32

In [11]:
prefix_cls = cur_class.split('_')[0]

checkpointer = ModelCheckpoint(filepath='./models/{0}.bestXception0419_alldata.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True, mode='val_acc')

try:
    # Fit the model
    history = model.fit_generator(datagen.flow(X, y, batch_size=batch_size),
                                  epochs=epochs, validation_data=(X_valid, y_valid),
                                  verbose=2, steps_per_epoch=X.shape[0] // batch_size,
                                  callbacks=[EarlyStopping(patience=5), checkpointer])
except KeyboardInterrupt:
    print('KeyboardInterrupt')

## 加载模型

In [7]:
df_test = pd.read_csv('../data/z_rank/Tests/question.csv', header=None)
df_test.columns = ['image_id', 'class', 'x']
del df_test['x']
df_test.head()

,image_id,class
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels
3,Images/collar_design_labels/4b09d4dca80caac42e...,collar_design_labels
4,Images/collar_design_labels/de91f00a05e84d7239...,collar_design_labels


In [8]:
df_load = df_test[(df_test['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

neckline_design_labels: 3147


,image_id,class
0,Images/neckline_design_labels/3d784813cda05081...,neckline_design_labels
1,Images/neckline_design_labels/0cb381e52f4b738a...,neckline_design_labels
2,Images/neckline_design_labels/e19268a3aa3c57ff...,neckline_design_labels
3,Images/neckline_design_labels/5a0b5522ae031a31...,neckline_design_labels
4,Images/neckline_design_labels/6e7c447b97b5017f...,neckline_design_labels


In [14]:
n = len(df_load)
X_test = np.zeros((n, width, width, 3), dtype=np.uint8)

for i in tqdm(range(n)):
    X_test[i] = cv2.resize(cv2.imread('../data/z_rank/{0}'.format(df_load['image_id'][i])), (width, width))

In [19]:
model.load_weights('./models/{}.bestXception0419_alldata.h5'.format(prefix_cls))

In [20]:
test_np = model.predict(X_test, batch_size=256)

In [15]:
result = []

for i, row in df_load.iterrows():
    tmp_list = test_np[i]
    tmp_result = ''
    for tmp_ret in tmp_list:
        tmp_result += '{:.4f};'.format(tmp_ret)
        
    result.append(tmp_result[:-1])

df_load['result'] = result
df_load.head()

In [23]:
df_load.to_csv('./result/{}_0419Xception_alldata.csv'.format(prefix_cls), header=None, index=False)